In [1]:
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# To get smooth animations
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')

In [2]:
import numpy as np
import os

In [3]:
import tensorflow as tf
from tensorflow import keras

# Saving a TensorFlow Model
## Using Tensorflow Serving
### Exporting SaveModels

Prepare data

In [4]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_new = X_test[:3]

Prepare and train a model

In [5]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

C:\Users\benso\anaconda3\envs\ml\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/10
1719/1719 [==============================] - 34s 17ms/step - loss: 0.7012 - accuracy: 0.8241 - val_loss: 0.3715 - val_accuracy: 0.9024
Epoch 2/10
1719/1719 [==============================] - 25s 15ms/step - loss: 0.3536 - accuracy: 0.9020 - val_loss: 0.2990 - val_accuracy: 0.9144
Epoch 3/10
1719/1719 [==============================] - 25s 15ms/step - loss: 0.3036 - accuracy: 0.9145 - val_loss: 0.2651 - val_accuracy: 0.9272
Epoch 4/10
1719/1719 [==============================] - 27s 16ms/step - loss: 0.2736 - accuracy: 0.9231 - val_loss: 0.2436 - val_accuracy: 0.9334
Epoch 5/10
1719/1719 [==============================] - 23s 14ms/step - loss: 0.2509 - accuracy: 0.9296 - val_loss: 0.2257 - val_accuracy: 0.9364
Epoch 6/10
1719/1719 [==============================] - 18s 10ms/step - loss: 0.2322 - accuracy: 0.9350 - val_loss: 0.2121 - val_accuracy: 0.9396
Epoch 7/10
1719/1719 [==============================] - 20s 12ms/step - loss: 0.2161 - accuracy: 0.9401 - val_loss: 0.1970 -

In [6]:
np.round(model.predict(X_new), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

In [7]:
# Save the model to Tensorflow SaveModel format
model_version = "0001"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)

# --- Legacy method (not be able to be load by keras)
# tf.saved_model.save(model, model_path)

# --- New method (can be load by keras)
keras.models.save_model(model, model_path)
# model.save(model_path)

INFO:tensorflow:Assets written to: my_mnist_model\0001\assets


In [8]:
# Load the model: 1. using tf.saved_model.load --> function/object
saved_model = tf.saved_model.load(model_path)

# Make prediction by simply make a function call
y_pred = saved_model(tf.constant(X_new, dtype=tf.float32))
print(y_pred)

tf.Tensor(
[[1.1424438e-04 1.5136631e-07 9.8006742e-04 2.7721331e-03 3.7535815e-06
  7.6288459e-05 3.9108013e-08 9.9556917e-01 5.3491975e-05 4.3077732e-04]
 [8.1708626e-04 3.5403158e-05 9.8826581e-01 7.0432499e-03 1.2935648e-07
  2.3344434e-04 2.5722249e-03 9.6381547e-10 1.0325175e-03 8.7983246e-08]
 [4.4414192e-05 9.7030061e-01 9.0606371e-03 2.2621900e-03 4.8723447e-04
  2.8747944e-03 2.2720499e-03 8.3591826e-03 4.0410073e-03 2.9790381e-04]], shape=(3, 10), dtype=float32)


In [9]:
# Load the model: 2. using keras.models.load_model --> keras model
model = keras.models.load_model(model_path)

# Make prediction as usual (it's a normal keras model object) 
y_pred = model.predict(tf.constant(X_new, dtype=tf.float32))
print(y_pred)

[[1.1424438e-04 1.5136631e-07 9.8006742e-04 2.7721331e-03 3.7535815e-06
  7.6288459e-05 3.9108013e-08 9.9556917e-01 5.3491975e-05 4.3077732e-04]
 [8.1708626e-04 3.5403158e-05 9.8826581e-01 7.0432499e-03 1.2935648e-07
  2.3344434e-04 2.5722249e-03 9.6381547e-10 1.0325175e-03 8.7983246e-08]
 [4.4414192e-05 9.7030061e-01 9.0606371e-03 2.2621900e-03 4.8723447e-04
  2.8747944e-03 2.2720499e-03 8.3591826e-03 4.0410073e-03 2.9790381e-04]]


In [10]:
!saved_model_cli show --dir {model_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_flatten_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          inputs

Test in command line

In [11]:
np.save("my_mnist_tests.npy", X_new)

In [13]:
!saved_model_cli run --dir {model_path} --tag_set serve --signature_def serving_default --inputs flatten_input=my_mnist_tests.npy

Result for output key dense_1:
[[1.1424438e-04 1.5136631e-07 9.8006742e-04 2.7721331e-03 3.7535815e-06
  7.6288459e-05 3.9108013e-08 9.9556917e-01 5.3491975e-05 4.3077732e-04]
 [8.1708626e-04 3.5403158e-05 9.8826581e-01 7.0432499e-03 1.2935648e-07
  2.3344434e-04 2.5722249e-03 9.6381547e-10 1.0325175e-03 8.7983246e-08]
 [4.4414192e-05 9.7030061e-01 9.0606371e-03 2.2621900e-03 4.8723447e-04
  2.8747944e-03 2.2720499e-03 8.3591826e-03 4.0410073e-03 2.9790381e-04]]


2022-02-07 10:24:43.255008: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-07 10:24:43.771093: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2147 MB memory:  -> device: 0, name: Quadro T2000, pci bus id: 0000:01:00.0, compute capability: 7.5
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from my_mnist_model\0001\variables\variables
